# 프롬프트 (`Prompt`)
`02_prompt.ipynb`

- LLM한테 주는 입력(지시, 맥락, 기억)
    - 지시: '~~해줘'
    - 맥락: Context - 현재 지시를 위해 제공하는 추가 정보
    - 기억: Memory - 지금까지 했던 대화 내용

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## `PromptTemplate`

- 단순히 1회성 명령을 내릴 때 사용

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model='gpt-5-nano', reasoning_effort="minimal")

In [3]:
# 추후에 체인.invoke에서 {} 내부에 들어갈 말을 채워줘야 함.
template = '{country}의 수도가 어디인가요?'

# .from_template 메서드로 프롬프트 만들기
prompt = PromptTemplate.from_template(template)

# {}를 채우는 방법 (우리는 결국 chain.invoke 로 쓰게 됨)
prompt.format(country='대한민국')

'대한민국의 수도가 어디인가요?'

In [4]:
chain = prompt | llm

chain.invoke({'country': '대한민국'}).content

'대한민국의 수도는 서울입니다.'

### Partial Variable (부분 변수)

- 프롬프트에서 매개변수 기본값 사용하기

In [5]:
template = '{c1}과 {c2}의 수도는 각각 어디인가요?'

prompt = PromptTemplate(
    template=template,
    input_variables=['c1'],
    partial_variables={
        'c2': '미국'
    }
)

prompt.format(c1='한국')
# prompt.format(c2='캐나다') 

'한국과 미국의 수도는 각각 어디인가요?'

In [6]:
chain = prompt | llm
chain.invoke({'c1': 'kor', 'c2': 'us'}).content

'한국어로 질문의 의도를 이렇게 이해했습니다: “kor과 us의 수도는 각각 어디인가요?”를 묻고 계신데, 보통 코드는 두 가지 해석이 있습니다.\n\n가능한 해석 1: 코어(KOR)와 미국(US)의 수도를 묻는 질문\n- KOR는 일반적으로 국가 코드로서의 한국(Korea)을 나타낼 때 사용되며, 해당 국가는 대한민국의 수도는 서울(Seoul)입니다.\n- US는 미국의 수도로 워싱턴 D.C.를 말합니다.\n\n가능한 해석 2: 다른 맥락의 약어일 가능성\n- 만약 “kor”나 “us”가 특정 프로그램, 데이터셋, 맥락에서 다른 약어로 쓰이는 경우, 그 맥락에 따른 수도가 있을 수 있습니다. 예를 들어 도시 이름이나 지역 코드의 약어일 수 있습니다. 이 경우 더 구체적인 맥락이 필요합니다.\n\n정리\n- 대한민국(KOR)의 수도: 서울\n- 미국(US)의 수도: 워싱턴 D.C.\n\n추가로 특정 맥락이 있다면 알려주시면 그에 맞춰 답변해 드리겠습니다.'

In [7]:
from datetime import datetime

def get_today():
    return datetime.now().strftime('%b %d')

prompt = PromptTemplate(
    template='오늘 날짜는 {today}입니다. 오늘 생일인 유명인 {n}명을 생년월일과 함께 나열해 주세요',
    input_variables=['n'],
    partial_variables={
        'today': get_today
    }
)

prompt.format(n=3)

'오늘 날짜는 Sep 01입니다. 오늘 생일인 유명인 3명을 생년월일과 함께 나열해 주세요'

In [8]:
llm = ChatOpenAI(model='gpt-5-nano')

chain = prompt | llm
# 오늘 날짜 기준
print(chain.invoke({'n': 3}).content)
print('===')
print(chain.invoke({'n': 3, 'today': 'Jan 2'}).content)

오늘이 9월 1일이므로 Sep 1에 생일인 유명인을 생년월일과 함께 알려드리겠습니다. 제 지식으로는 확실한 한 명만 확인 가능하므로, 먼저 한 분을 제시하고 나머지 두 분은 인터넷 검색으로 확인해 정확하게 드리겠습니다.

- Zendaya — 1996년 9월 1일

나머지 두 명도 확인해서 같이 드릴까요? 검색 허용해 주시면 바로 확실한 3명을 생년월일과 함께 정리해 드리겠습니다.
===
요청하신 대로 Jan 2에 생일이 있는 유명인을 생년월일과 함께 3명 나열하려면 정확한 정보를 확인해야 합니다. 현재 제가 확실히 기억하는 한 명은:

- Kate Bosworth — 1983년 1월 2일

나머지 두 명은 출처에 따라 생일 정보가 다르게 기재될 수 있어, 정확하게 세 명을 확정하려면 인터넷 검색으로 확인이 필요합니다. 원하시면 인터넷 검색으로 오늘 생일인 다른 두 명까지 포함해 정확한 목록(생년월일과 함께)로 바로 제공해 드리겠습니다. 진행할까요?


## `ChatPromptTemplate`
- 채팅을 주고받는 템플릿 생성용
- 대화 목록을 LLM에게 주입
- 하나의 Chat 은 `role` 과 `message` 로 구성됨

In [9]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template('{country}의 수도는 어디?')
chat_prompt.format(country='한국')

'Human: 한국의 수도는 어디?'

In [10]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role - message
        ('system', '당신은 친절한 AI어시스트. 이름은 {name} 야.'),
        ('human', '반가워!'),
        ('ai', '무엇을 도와드릴까요?'),
        ('human', '{user_input}')
    ]
)

# format vs format_messages
messages_str = chat_template.format(name='gaida', user_input='이름이 뭐니?')
messages_cls = chat_template.format_messages(name='gaida', user_input='이름이 뭐니?')
print(messages_str)
print(messages_cls)

System: 당신은 친절한 AI어시스트. 이름은 gaida 야.
Human: 반가워!
AI: 무엇을 도와드릴까요?
Human: 이름이 뭐니?
[SystemMessage(content='당신은 친절한 AI어시스트. 이름은 gaida 야.', additional_kwargs={}, response_metadata={}), HumanMessage(content='반가워!', additional_kwargs={}, response_metadata={}), AIMessage(content='무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='이름이 뭐니?', additional_kwargs={}, response_metadata={})]


In [17]:
# 한덩어리 텍스트
llm.invoke(messages_str).content
# 실제 대화 내역으로 인지 (Langsmith 가서 확인)
llm.invoke(messages_cls).content

'안녕하세요! 저는 gaida라고 해요. 반가워요!'

In [16]:
chain = chat_template | llm | StrOutputParser()

chain.invoke({'name': '가이다', 'user_input': '내 이름은 유태영. 너와 나의 이름점을 봐줘'})

"반가워요, 유태영님! 이름점을 재미로 살펴볼게요.\n\n유태영이라는 이름을 보면,  \n'유'는 넓고 포용력 있는 성격을,  \n'태'는 크고 위대함을,  \n'영'은 영리하고 활기찬 성격을 나타낼 수 있어요.\n\n이름 속에 담긴 의미와 함께, 유태영님은 사람들과 잘 어울리고, 성장하는 데 관심이 많으실 것 같네요!  \n\n혹시 더 궁금한 점이나 분석을 원하시면 알려주세요!"

## 프롬프팅 방법
내가 원하는 답변을 구구절절 설명 X -> 예시를 주고 답변을 형태를 유도
- Zero-shot prompting : 예시 없이 질문만 -> 바로 답변
- One-shot prompting  : 예시 1개 + 질문 -> 모델이 예시를 모방해서 답변
- **Few-shot prompting**  : 예시 여러개 + 질문 -> 예시들의 패턴을 일반화 해서 답변

In [13]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model='gpt-4.1-nano')

examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]

example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"

chain = prompt | llm | StrOutputParser()

res = chain.invoke({'question': question})

print(res)

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: Google이 창립된 연도는 언제인가요?  
중간 답변: 구글은 1998년에 창립되었습니다.  
추가 질문: Bill Gates는 1998년 당시 몇 살이었나요?  
중간 답변: Bill Gates는 1998년 당시 43세였습니다. (그는 1955년 10월 28일생입니다.)  
최종 답변은: 43살


## `langchain-hub`
다양한 사용자들이 업로드한 프롬프트를 받아서 활용

In [14]:
from langchain import hub

prompt = hub.pull('hwchase17/react')
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [15]:
prompt = PromptTemplate.from_template(
    '내 첫 프롬프트 Push!'
)

hub.push('gaida1st/prompt-hub-test', prompt)

LangSmithUserError: Cannot create a prompt for another tenant.
Current tenant: None,
Requested tenant: gaida1st